<a href="https://colab.research.google.com/github/eduardokern/ML/blob/yolo/notebook/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Treinamento da rede YOLO

Projeto de criação de uma base de dados e treinamento da rede YOLO .

Seguindo os exemplos de aula, vamos rotular uma base de dados e aplicar o treinamento com a rede YOLO.

Para essa tarefa será necessário utilizar o software LabelMe: http://labelme.csail.mit.edu/Release3.0/ para rotular as imagens.

Também será necessário utilizar a rede YOLO, disponível em: https://pjreddie.com/darknet/yolo/.

Para quem preferir e não quiser rotular uma base de dados, pode usar as imagens já rotuladas do COCO: https://cocodataset.org/#home.

E para quem estiver utilizando um computador que não consiga rodar a rede YOLO, pode utilizar o transfer learning no COLAB: https://colab.research.google.com/drive/1lTGZsfMaGUpBG4inDIQwIJVW476ibXk_#scrollTo=j0t221djS1Gk.

O trabalho deve conter pelo menos duas classes retreinadas para detecção, além das classes já treinadas previamente antes de realizar o transfer learning.  

## Check CUDA release version

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version

## Install cuDNN according to the current CUDA version

In [ ]:
#!sudo apt-get -y install cudnn9-cuda-12

!wget https://developer.download.nvidia.com/compute/cudnn/redist/cudnn/linux-x86_64/cudnn-linux-x86_64-9.7.0.66_cuda12-archive.tar.xz

# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xf cudnn-linux-x86_64-9.7.0.66_cuda12-archive.tar.xz --totals

!cp -vnpr cudnn-linux-x86_64-9.7.0.66_cuda12-archive/include/* /usr/local/cuda/include
!cp -vnpr cudnn-linux-x86_64-9.7.0.66_cuda12-archive/lib /usr/local/cuda

!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

!rm cudnn-linux-x86_64-9.7.0.66_cuda12-archive.tar.xz
!rm -rf cudnn-linux-x86_64-9.7.0.66_cuda12-archive

## Installing Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet/
%cd /content/darknet

#Compile Darknet
!make CFLAGS+=-w

## Some Utils

* imgShow() - Will help us to show an image in the remote VM

In [ ]:
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Testing if building process worked

In [ ]:
# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

# Training YOLO on a different dataset

In [ ]:
%cd /content/data
!wget -c https://pjreddie.com/media/files/coco/labels.tgz
!tar xf labels.tgz


In [ ]:
%cd /content
!rm -rf coco
!mkdir coco
%cd coco
!mkdir images
%cd images

#Download COCO images
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip -q train2014.zip
!rm train2014.zip

!wget http://images.cocodataset.org/zips/val2014.zip
!unzip -q val2014.zip
!rm val2014.zip

!wget http://images.cocodataset.org/zips/test2014.zip
!unzip -q test2014.zip
!rm test2014.zip

%cd ..

# Download COCO Metadata
!wget -c https://pjreddie.com/media/files/instances_train-val2014.zip
!unzip -q instances_train-val2014.zip

!wget -c https://pjreddie.com/media/files/coco/labels.tgz
!tar xf labels.tgz

!wget -c https://pjreddie.com/media/files/coco/5k.part
!wget -c https://pjreddie.com/media/files/coco/trainvalno5k.part

# Set Up Image Lists
!paste <(awk "{print \"$PWD\"}" <5k.part) 5k.part | tr -d '\t' > 5k.txt
!paste <(awk "{print \"$PWD\"}" <trainvalno5k.part) trainvalno5k.part | tr -d '\t' > trainvalno5k.txt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!/content/darknet/darknet detector train "/content/drive/MyDrive/Colab/ML/yolo/coco.data" "/content/drive/MyDrive/Colab/ML/yolo/yolov3.cfg" "/content/drive/MyDrive/Colab/ML/yolo/darknet53.conv.74" -dont_show

In [ ]:
# Execute darknet using YOLOv3 model with new-trained weights to detect objects on a test image
!/content/darknet/darknet detect /content/drive/MyDrive/Colab/ML/yolo/yolov3-detect.cfg ./_last.weights /content/coco/images/test2014/COCO_test2014_000000000001 -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')